In [1]:
import ga4gh.client
from __future__ import print_function
baseURL = "http://localhost:8080/ga4gh"
client = ga4gh.client.HttpClient(baseURL)

datasets = list(client.search_datasets())

# search reference sets
# WyJHUkNoMzciXQ  GRCh37
# WyJodW1hbl9nMWtfdjM3X2RlY295Il0 human_g1k_v37_decoy <---
reference_set = list(client.search_reference_sets())[1]

# search references [chromosomes] for human_g1k_v37_decoy
# WyJHUkNoMzciLCIxIl0     1
# WyJHUkNoMzciLCIyIl0     2
# WyJHUkNoMzciLCIzIl0     3
# WyJodW1hbl9nMWtfdjM3X2RlY295IiwiMSJd    1
# WyJodW1hbl9nMWtfdjM3X2RlY295IiwiMiJd    2
# WyJodW1hbl9nMWtfdjM3X2RlY295IiwiMyJd    3 <---
# WyJodW1hbl9nMWtfdjM3X2RlY295IiwiNCJd    4
# WyJodW1hbl9nMWtfdjM3X2RlY295IiwiNSJd    5 <---
references = list(client.search_references(reference_set_id=reference_set.id))

# WyIxa2ctcDMtc3Vic2V0Il0 1kg-p3-subset
# WyJ5dWVmdXNpb25zIl0     yuefusions <---
dataset = client.get_dataset(datasets[1].id)

# WyJ5dWVmdXNpb25zIiwicmdzIiwicmVhZHRocm91Z2hfdGVzdCJd    readthrough_test <---
read_group_sets = list(client.search_read_group_sets(dataset_id=dataset.id))
read_group_set = read_group_sets[0]

def reads_at_breakpoint(read_group_set, reference, position):
    """
    Returns a list of reads from a read_group_set at a poisition in a reference.

    :param :class:`ga4gh.reads_pb2.ReadGroupSet` read_group_set The read_group_set to search for reads.
    :param :class:`ga4gh.references_pb2.Reference` reference The reference for for the position.
    :param int position The coordinates for the position.

    :return: A list of reads.
    :rtype: list
    """
    for read_group in read_group_set.read_groups:

        sequences = list(client.search_reads(read_group_ids=[read_group.id], 
                                            start=position, end=position+1,
                                            reference_id=reference.id))

        return sequences

def readthrough_fusion_support(read_group_set, reference, breakpoint_1, breakpoint_2):
    """
    Returns the number of reads that support a given readthrough fusion event for a given read_group_set.

    :param :class:`ga4gh.reads_pb2.ReadGroupSet` read_group_set The read_group_set to search for reads.
    :param :class:`ga4gh.references_pb2.Reference` reference The reference for the readthrough fusion event.
    :param int breakpoint_1 The first breakpoint for the readthrough fusion event.
    :param int breakpoint_2 The second breakpoint for the readthrough fusion event.

    :return: A set of fragment_names that support the given readthrough fusion event.
    :rtype: set
    """
    breakpoint_1_reads = set([read.fragment_name for read in reads_at_breakpoint(read_group_set, reference, breakpoint_1)])
    breakpoint_2_reads = set([read.fragment_name for read in reads_at_breakpoint(read_group_set, reference, breakpoint_2)])

    intersection_reads = breakpoint_1_reads & breakpoint_2_reads
    
    return intersection_reads

# chr3	15604865	chr3	15531144	HACL1	COLQ
# returns: 8 supporting reads
print(len(readthrough_fusion_support(read_group_set, references[2], 15604865, 15531144)))

# chr5	40764616	chr5	40747121	PRKAA1	TTC33
# returns: 9 supporting reads
print(len(readthrough_fusion_support(read_group_set, references[4], 40764616, 40747121)))

# no fusion event
# returns: 0 supporting reads
print(len(readthrough_fusion_support(read_group_set, references[4], 15604865, 123)))

8
9
0
